In [1]:
import pandas
import re, json
import csv

import torch
import torch.nn as nn
from datasets import load_metric,Dataset,DatasetDict
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer

import evaluate

import numpy as np
import nltk
import os
import random
from sklearn.model_selection import train_test_split
from typing import List, Optional, Tuple, Union, Dict, Any

In [2]:
data = open('annotated_capstone_data.csv')

In [3]:
reader = csv.DictReader(data)
myList = list()
for dictionary in reader:
    myList.append(dictionary)

In [4]:
dialogues = []
gold_sum = []
generated_sum = []
hal_tags = []

for entry in myList[:100]:
    dialogues.append(entry['Dialogue'].strip())
    gold_sum.append(entry['Reference Summary'].strip())
    generated_sum.append(entry['Generated Summary'].strip())
    hal_tags.append(entry['Annotations'].strip())

In [5]:
train = Dataset.from_dict({'dialogue': dialogues, 'summary':generated_sum, 'tags':hal_tags})

In [6]:
raw_data = DatasetDict({'train':train})

In [7]:
raw_data

DatasetDict({
    train: Dataset({
        features: ['dialogue', 'summary', 'tags'],
        num_rows: 100
    })
})

In [9]:
model_checkpoint = "facebook/bart-large"

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
 with tokenizer.as_target_tokenizer():
     labels = tokenizer(generated_sum[0], max_length=128, truncation=True)

In [23]:
generated_sum[0]

"doesn't have Betty's number. Amanda suggests Hannah to ask Larry."

In [15]:
labels

{'input_ids': [0, 34318, 75, 33, 16666, 18, 346, 4, 10641, 3649, 11029, 7, 1394, 6045, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
tokenizer.decode(labels['input_ids'][-2])

'.'